In [1]:
import numpy as np
import pandas as pd
import math
import itertools
import torch

# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")


In [2]:
from google.colab import drive
drive.mount('/content/drive')


pd.set_option('display.max_columns', None)

Mounted at /content/drive


# 1. Extract Dataset

In [3]:
# Define the path to the file
file_path = '/content/drive/MyDrive/Colab Notebooks/ml-1m/cleaned_movies.csv'

movies_df = pd.read_csv(file_path,index_col=0)

movies_df.head()

,movieId,title,Release_Year,Language_Castilian,Language_Chinese,Language_English,Language_Flemish,Language_French,Language_German,Language_Italian,Language_Japanese,Language_Persian,Language_Swedish,Language_others,director_Alfred Hitchcock,director_Barry Levinson,director_Clint Eastwood,director_John Carpenter,director_Robert Stevenson,director_Sidney Lumet,director_Spike Lee,director_Stanley Kubrick,director_Steven Spielberg,director_Woody Allen,director_others,Genre_Drama,Genre_Comedy,Genre_Action,Genre_Thriller,Genre_Romance,Genre_Horror,Genre_Adventure,Genre_Sci-Fi,Genre_Children's,Genre_Crime,Star_robert de niro,Star_whoopi goldberg,Star_harvey keitel,Star_bruce willis,Star_robert duvall,Star_gene hackman,Star_jack nicholson,Star_samuel l jackson,Star_sean connery,Star_demi moore
0,1,Toy Story (1995),1995,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1,2,Jumanji (1995),1995,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False
2,3,Grumpier Old Men (1995),1995,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,4,Waiting to Exhale (1995),1995,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,5,Father of the Bride Part II (1995),1995,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [4]:
movies_df.isnull().sum()

movieId                      0
title                        0
Release_Year                 0
Language_Castilian           0
Language_Chinese             0
Language_English             0
Language_Flemish             0
Language_French              0
Language_German              0
Language_Italian             0
Language_Japanese            0
Language_Persian             0
Language_Swedish             0
Language_others              0
director_Alfred Hitchcock    0
director_Barry Levinson      0
director_Clint Eastwood      0
director_John Carpenter      0
director_Robert Stevenson    0
director_Sidney Lumet        0
director_Spike Lee           0
director_Stanley Kubrick     0
director_Steven Spielberg    0
director_Woody Allen         0
director_others              0
Genre_Drama                  0
Genre_Comedy                 0
Genre_Action                 0
Genre_Thriller               0
Genre_Romance                0
Genre_Horror                 0
Genre_Adventure              0
Genre_Sc

In [5]:
file_path = '/content/drive/MyDrive/Colab Notebooks/ml-1m/users.dat'

# Define the column names
column_names = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code']

# Read the file into a DataFrame
users_df = pd.read_csv(file_path, sep='::', names=column_names, engine='python')

users_df.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [6]:
users_df.isnull().sum()

UserID        0
Gender        0
Age           0
Occupation    0
Zip-code      0
dtype: int64

In [7]:
file_path = '/content/drive/MyDrive/Colab Notebooks/ml-1m/ratings.dat'

# Define the column names
column_names = ['UserID', 'MovieID', 'Rating', 'Timestamp']

# Read the file into a DataFrame
ratings_df = pd.read_csv(file_path, sep='::', names=column_names, engine='python')

# Convert the Timestamp column to datetime
ratings_df['Timestamp'] = pd.to_datetime(ratings_df['Timestamp'], unit='s')

# remove records that the movieid or userid has no details
ratings_df = ratings_df[ratings_df['MovieID'].isin(list(movies_df['movieId'].unique()))]
ratings_df = ratings_df[ratings_df['UserID'].isin(list(users_df['UserID'].unique()))]

# Display the first few rows of the DataFrame
ratings_df.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,2000-12-31 22:12:40
1,1,661,3,2000-12-31 22:35:09
2,1,914,3,2000-12-31 22:32:48
3,1,3408,4,2000-12-31 22:04:35
4,1,2355,5,2001-01-06 23:38:11


In [8]:
ratings_df.isnull().sum()

UserID       0
MovieID      0
Rating       0
Timestamp    0
dtype: int64

In [9]:
!pip3 install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.3.0+cu121.html
!pip3 install torch_geometric

Looking in links: https://data.pyg.org/whl/torch-2.3.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 947.1/947.1 kB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.1 MB/s eta 0:00:00


In [10]:
import torch
import torch.nn.functional as F
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from torch_geometric.data import HeteroData
from torch_geometric.transforms import ToUndirected, RandomLinkSplit
from transformers import BertTokenizer, BertModel
import torch.optim as optim
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import SAGEConv, to_hetero
from torch import Tensor
from torch_geometric.data import HeteroData

# 2. Preprocess Data

In [11]:
!pip3 install pyzipcode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyzipcode: filename=pyzipcode-3.0.1-py3-none-any.whl size=1932161 sha256=0c0bd57147fdc0e4e11297fbe150f2721edb96c5e673552400821b21cff68f4e
  Stored in directory: /root/.cache/pip/wheels/33/11/bb/75313afc9ef5609e3997d374a6f6c4d48932fdb8ed8fe2ea77
Successfully built pyzipcode


In [12]:
# One-hot encode Gender and Occupation
users_df = pd.get_dummies(users_df, columns=['Gender', 'Occupation'], drop_first=True)

# # Standardize Age
# scaler = StandardScaler()
# users_df[['Age']] = scaler.fit_transform(users_df[['Age']])


# Convert Zipcode to State and One-Hot encode State
from pyzipcode import ZipCodeDatabase

def zipcode_converter(zipcode):
    zcdb = ZipCodeDatabase()
    try:
        return zcdb[int(zipcode)].state
    except:
        return 'None'


users_df['State'] = users_df['Zip-code'].apply(lambda x: zipcode_converter(x))
users_df = pd.get_dummies(users_df, columns=['State'], drop_first=True)

# Drop UserID and original Zip-code columns
users_df = users_df.drop(columns=['UserID', 'Zip-code'])

users_df

,Age,Gender_M,Occupation_1,Occupation_2,Occupation_3,Occupation_4,Occupation_5,Occupation_6,Occupation_7,Occupation_8,Occupation_9,Occupation_10,Occupation_11,Occupation_12,Occupation_13,Occupation_14,Occupation_15,Occupation_16,Occupation_17,Occupation_18,Occupation_19,Occupation_20,State_AL,State_AR,State_AZ,State_CA,State_CO,State_CT,State_DC,State_DE,State_FL,State_GA,State_HI,State_IA,State_ID,State_IL,State_IN,State_KS,State_KY,State_LA,State_MA,State_MD,State_ME,State_MI,State_MN,State_MO,State_MS,State_MT,State_NC,State_ND,State_NE,State_NH,State_NJ,State_NM,State_NV,State_NY,State_None,State_OH,State_OK,State_OR,State_PA,State_PR,State_RI,State_SC,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY
0,1,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,56,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,25,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,45,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,25,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,25,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6036,45,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,

In [13]:
users_df.columns

Index(['Age', 'Gender_M', 'Occupation_1', 'Occupation_2', 'Occupation_3',
       'Occupation_4', 'Occupation_5', 'Occupation_6', 'Occupation_7',
       'Occupation_8', 'Occupation_9', 'Occupation_10', 'Occupation_11',
       'Occupation_12', 'Occupation_13', 'Occupation_14', 'Occupation_15',
       'Occupation_16', 'Occupation_17', 'Occupation_18', 'Occupation_19',
       'Occupation_20', 'State_AL', 'State_AR', 'State_AZ', 'State_CA',
       'State_CO', 'State_CT', 'State_DC', 'State_DE', 'State_FL', 'State_GA',
       'State_HI', 'State_IA', 'State_ID', 'State_IL', 'State_IN', 'State_KS',
       'State_KY', 'State_LA', 'State_MA', 'State_MD', 'State_ME', 'State_MI',
       'State_MN', 'State_MO', 'State_MS', 'State_MT', 'State_NC', 'State_ND',
       'State_NE', 'State_NH', 'State_NJ', 'State_NM', 'State_NV', 'State_NY',
       'State_None', 'State_OH', 'State_OK', 'State_OR', 'State_PA',
       'State_PR', 'State_RI', 'State_SC', 'State_SD', 'State_TN', 'State_TX',
       'State_UT'

In [14]:
#convert pandas dataframe to tensor
user_features = torch.tensor(users_df.values.astype('float32'), dtype=torch.float32)

In [15]:
# Extract movie features (excluding MovieID, Title)
movie_features = movies_df.drop(columns=['movieId', 'title'])

movie_features

,Release_Year,Language_Castilian,Language_Chinese,Language_English,Language_Flemish,Language_French,Language_German,Language_Italian,Language_Japanese,Language_Persian,Language_Swedish,Language_others,director_Alfred Hitchcock,director_Barry Levinson,director_Clint Eastwood,director_John Carpenter,director_Robert Stevenson,director_Sidney Lumet,director_Spike Lee,director_Stanley Kubrick,director_Steven Spielberg,director_Woody Allen,director_others,Genre_Drama,Genre_Comedy,Genre_Action,Genre_Thriller,Genre_Romance,Genre_Horror,Genre_Adventure,Genre_Sci-Fi,Genre_Children's,Genre_Crime,Star_robert de niro,Star_whoopi goldberg,Star_harvey keitel,Star_bruce willis,Star_robert duvall,Star_gene hackman,Star_jack nicholson,Star_samuel l jackson,Star_sean connery,Star_demi moore
0,1995,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1,1995,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False
2,1995,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,1995,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,1995,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3877,1971,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3878,2000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
3879,2000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3880,2000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [16]:
movie_features.columns

Index(['Release_Year', 'Language_Castilian', 'Language_Chinese',
       'Language_English', 'Language_Flemish', 'Language_French',
       'Language_German', 'Language_Italian', 'Language_Japanese',
       'Language_Persian', 'Language_Swedish', 'Language_others',
       'director_Alfred Hitchcock', 'director_Barry Levinson',
       'director_Clint Eastwood', 'director_John Carpenter',
       'director_Robert Stevenson', 'director_Sidney Lumet',
       'director_Spike Lee', 'director_Stanley Kubrick',
       'director_Steven Spielberg', 'director_Woody Allen', 'director_others',
       'Genre_Drama', 'Genre_Comedy', 'Genre_Action', 'Genre_Thriller',
       'Genre_Romance', 'Genre_Horror', 'Genre_Adventure', 'Genre_Sci-Fi',
       'Genre_Children's', 'Genre_Crime', 'Star_robert de niro',
       'Star_whoopi goldberg', 'Star_harvey keitel', 'Star_bruce willis',
       'Star_robert duvall', 'Star_gene hackman', 'Star_jack nicholson',
       'Star_samuel l jackson', 'Star_sean connery', 'St

In [17]:
#convert pandas dataframe to tensor
movie_features = torch.tensor(movie_features.values.astype('float32'), dtype=torch.float32)

# 3. Construct Graph Data

In [18]:
user_mapping = {idx: i for i, idx in enumerate(ratings_df['UserID'].unique())}
movie_mapping = {idx: i for i, idx in enumerate(ratings_df['MovieID'].unique())}

In [19]:
src = [user_mapping[idx] for idx in ratings_df['UserID']]
dst = [movie_mapping[idx] for idx in ratings_df['MovieID']]
edge_index = torch.tensor([src, dst])
edge_label = torch.from_numpy(ratings_df['Rating'].values).to(torch.long)

In [20]:
data = HeteroData()
# # Save node indices:
data["user"].node_id = torch.arange(len(user_mapping))
data["movie"].node_id = torch.arange(len(movies_df))
# Add the node features and edge indices:
data["user"].x = user_features
data["movie"].x = movie_features
data["user", "rates", "movie"].edge_index = edge_index
data["user", "rates", "movie"].edge_label = edge_label
# We also need to make sure to add the reverse edges from movies to users
# in order to let a GNN be able to pass messages in both directions.
# We can leverage the `T.ToUndirected()` transform for this from PyG:
data = ToUndirected()(data)

In [21]:
data

HeteroData(
  user={
    node_id=[6040],
    x=[6040, 74],
  },
  movie={
    node_id=[3435],
    x=[3435, 43],
  },
  (user, rates, movie)={
    edge_index=[2, 976271],
    edge_label=[976271],
  },
  (movie, rev_rates, user)={
    edge_index=[2, 976271],
    edge_label=[976271],
  }
)

# 4. Train Model

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
# For this, we first split the set of edges into
# training (80%), validation (10%), and testing edges (10%).
# Across the training edges, we use 70% of edges for message passing, and 30% of edges for supervision.
# We further want to generate fixed negative edges for evaluation with a ratio of 2:1 negative edges during training will be generated on-the-fly.
# We can leverage the `RandomLinkSplit()` transform for this from PyG:
transform = RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    # disjoint_train_ratio=0.3,
    neg_sampling_ratio=0.0,
    add_negative_train_samples=False,
    edge_types=("user", "rates", "movie"),
    rev_edge_types=("movie", "rev_rates", "user"),
)
train_data, val_data, test_data = transform(data)


In [24]:
# We have an unbalanced dataset with many labels for rating 3 and 4, and very few for 0 and 1. Therefore we use a weighted MSE loss.
# Count the frequency of each value in an array of non-negative ints: https://pytorch.org/docs/stable/generated/torch.bincount.html
weight = torch.bincount(train_data['user', 'movie'].edge_label)
# Take the maximum number of appearences of a rate and normalize all with that number
weight = weight.max() / weight

In [25]:
def weighted_mse_loss(pred, target, weight=None):
    weight = 1. if weight is None else weight[target].to(pred.dtype)
    return (weight * (pred - target.to(pred.dtype)).pow(2)).mean()


In [26]:
from torch.nn import Linear

class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['user'][row], z_dict['movie'][col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)


model = Model(hidden_channels=32).to(device)

In [27]:
# run one model step so the number of parameters can be inferred:
with torch.no_grad():
    model.encoder(train_data.x_dict, train_data.edge_index_dict)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [28]:
def train():
    model.train()
    optimizer.zero_grad()

    '''
    train_data.x_dict = dictionary that contains the initial embeddings of both the users and the movies
    train_data.edge_index_dict = dictionary containing connections between nodes (contains connection types)
    '''

    pred = model(train_data.x_dict, train_data.edge_index_dict, train_data['user', 'movie'].edge_label_index)
    target = train_data['user', 'movie'].edge_label

    loss = weighted_mse_loss(pred, target, weight)
    loss.backward()
    optimizer.step()
    return float(loss)


In [29]:
def train_model(data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict, data['user', 'movie'].edge_label_index)
    pred = pred.clamp(min=0, max=5)
    target = data['user', 'movie'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()

    return float(rmse)

In [30]:
from sklearn.metrics import roc_auc_score

no_improvement = 0
tolerance = 5
best_validate_rmse = 100

for epoch in range(1, 50):
    loss = train()
    train_rmse = train_model(train_data)
    val_rmse = train_model(val_data)

    print(f'Epoch: {epoch:03d}, Loss: {loss:.3f}, Train RMSE: {train_rmse:.3f}, Val RMSE: {val_rmse:.3f}')

    if epoch >= 20:
      if val_rmse < best_validate_rmse:
          best_validate_rmse = val_rmse
          no_improvement = 0
      else:
          no_improvement += 1

    # Check if we need to stop training
    if no_improvement >= tolerance:
        print(f'Early stopping at epoch {epoch+1} as validation loss did not improve for {tolerance} consecutive epochs.')
        break


Epoch: 001, Loss: 2507.199, Train RMSE: 3.755, Val RMSE: 3.748
Epoch: 002, Loss: 24407.055, Train RMSE: 3.755, Val RMSE: 3.748
Epoch: 003, Loss: 618.303, Train RMSE: 1.802, Val RMSE: 1.810
Epoch: 004, Loss: 3567.205, Train RMSE: 1.802, Val RMSE: 1.810
Epoch: 005, Loss: 3633.012, Train RMSE: 1.802, Val RMSE: 1.810
Epoch: 006, Loss: 1132.667, Train RMSE: 1.724, Val RMSE: 1.720
Epoch: 007, Loss: 4.420, Train RMSE: 3.755, Val RMSE: 3.748
Epoch: 008, Loss: 769.668, Train RMSE: 3.755, Val RMSE: 3.748
Epoch: 009, Loss: 1320.796, Train RMSE: 3.755, Val RMSE: 3.748
Epoch: 010, Loss: 1085.176, Train RMSE: 3.755, Val RMSE: 3.748
Epoch: 011, Loss: 412.296, Train RMSE: 3.755, Val RMSE: 3.748
Epoch: 012, Loss: 22.491, Train RMSE: 1.802, Val RMSE: 1.810
Epoch: 013, Loss: 160.545, Train RMSE: 1.802, Val RMSE: 1.810
Epoch: 014, Loss: 483.411, Train RMSE: 1.802, Val RMSE: 1.810
Epoch: 015, Loss: 403.816, Train RMSE: 1.802, Val RMSE: 1.810
Epoch: 016, Loss: 201.014, Train RMSE: 1.802, Val RMSE: 1.810
Epo

In [34]:
PATH = '/content/drive/MyDrive/Colab Notebooks/model'
torch.save(model.state_dict(), PATH)

In [35]:
def test(data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict, data['user', 'movie'].edge_label_index)
    pred = pred.clamp(min=0, max=5)
    target = data['user', 'movie'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse), pred, target

In [36]:
test_rmse, test_pred, test_target = test(test_data)

In [37]:
from sklearn.metrics import precision_recall_fscore_support,accuracy_score

# test_pred = test_pred.tolist()
# test_target = test_target.tolist()
threshold = 3
pred_result = []
target_result = []

for i in range(len(test_pred)):
    if test_pred[i]>= threshold:
        pred_result.append(1)
    else:
        pred_result.append(0)

    if test_target[i]>= threshold:
        target_result.append(1)
    else:
        target_result.append(0)

accuracy = accuracy_score(target_result, pred_result)
matrix = precision_recall_fscore_support(target_result, pred_result, average='macro')

print(f'Test RMSE is: {test_rmse:03f}, Accuracy: {accuracy: 03f}, Precision: {matrix[0]: 03f}, Recall: {matrix[1]: 03f}, F-Score: {matrix[2]: 03f}')

Test RMSE is: 1.804534, Accuracy:  0.836449, Precision:  0.418224, Recall:  0.500000, F-Score:  0.455471


# 5. Make a Recommendation

In [43]:
model = torch.load(PATH)

In [38]:
user_profile = {
    "UserID": 10,
    "Gender": "M",
    "Age": 25,
    "Occupation": 12,  # Programmer
    "Zip-code": "94303"
}

In [39]:
movie_profile = {
    "Release_Year_Range": [1975,2000],
    "Language": ["English"],
    "director": ["Steven Spielberg"],
    "Genre": ["Action","Horror"],
    "Star": ['roy scheider', 'robert shaw', 'richard dreyfuss', 'lorraine gary', 'murray hamilton', 'carl gottlieb', 'jeffrey kramer', 'susan backlinie', 'jonathan filley'],
    # "Keywords": ['based on novel or book', 'beach', 'fishing', 'atlantic ocean', 'bathing', 'shipwreck', 'shark attack', 'police chief', 'ferry boat', 'dying and death', 'animal attack', 'long island, new york', 'dead child', 'creature', 'skinny dipping', 'shark', 'great white shark', 'dead dog', 'child killed by animal', 'fourth of july', 'severed leg', 'fishing boat', 'animal horror', 'shark cage']
}

In [40]:
def get_movie_recommendation(user_profile, movie_profile):
    num_movies = len(data['movie'].x)
    user_id = user_profile['UserID']
    movies_features = list(movies_df.columns)
    movie_mapping = {i: idx for i, idx in enumerate(movies_df.index)}

    row = torch.tensor([user_id] * num_movies)
    col = torch.arange(num_movies)
    edge_label_index = torch.stack([row, col], dim=0)

    pred = model(data.x_dict, data.edge_index_dict, edge_label_index)
    pred = pred.clamp(min=0, max=5)

    idx_max = torch.topk(pred, 50).indices
    # check if how many features each recommended movie align with user's preference
    movies = []
    preferred_movie_features = []
    for key in movie_profile.keys():
        if key != 'Release_Year_Range':
            for item in movie_profile[key]:
                preferred_movie_features.append(key+'_'+item)

    for i in idx_max:
        movie = {}
        movieId = movie_mapping[int(i)]
        movie['MovieID'] = movieId

        movie['title'] = movies_df.loc[movieId].title

        movie['satisfied_features'] = 0
        if movies_df.loc[movieId].Release_Year in movie_profile['Release_Year_Range']:
            movie['satisfied_features'] += 1

        for preferred_movie_feature in preferred_movie_features:
            if preferred_movie_feature in movies_features:
                if movies_df[preferred_movie_feature].loc[movieId] == True:
                    movie['satisfied_features'] += 1

        movies.append(movie)

    #get top 5 movies
    top5_movies = sorted(movies, key=lambda d: d['satisfied_features'],reverse=True)[:5]
    print('Recommended movies for userId ' + str(user_id))
    for top_movie in top5_movies:
        print(top_movie['title'])

    return top5_movies

In [41]:
top5_movies = get_movie_recommendation(user_profile,movie_profile)

Recommended movies for userId 10
Dead Presidents (1995)
Sudden Death (1995)
Mortal Kombat (1995)
Get Shorty (1995)
Heat (1995)


In [42]:
top5_movies

[{'MovieID': 41, 'title': 'Dead Presidents (1995)', 'satisfied_features': 2},
 {'MovieID': 8, 'title': 'Sudden Death (1995)', 'satisfied_features': 2},
 {'MovieID': 43, 'title': 'Mortal Kombat (1995)', 'satisfied_features': 2},
 {'MovieID': 20, 'title': 'Get Shorty (1995)', 'satisfied_features': 2},
 {'MovieID': 5, 'title': 'Heat (1995)', 'satisfied_features': 2}]